In [1]:
import pandas as pd
import numpy as np
import io
import os
import re
import string
import pymorphy2
from collections import OrderedDict
from nltk.stem.snowball import SnowballStemmer
from stop_words import get_stop_words
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt

Гиперпараметры:

In [2]:
min_word_length = 2

Читаем директорию с файлами каталога:

In [3]:
list_of_files = os.listdir('data/top_ym/')
files_num = len(list_of_files)

Выбор необходимых атрибутов:

In [4]:
market_features = ['Категория', 'Производитель', 'Название модели']

Читаем файлы с необходимыми атрибутами в один фрейм:

In [5]:
df = pd.DataFrame()

for item in tqdm(list_of_files):
    df_file = pd.read_csv(('data/top_ym/' + str(item)), delimiter=';', encoding='windows-1251', usecols=market_features)
    df = pd.concat([df, df_file], axis=0)

Записываем фрейм в csv:

In [6]:
df.to_csv('data/market_dirty.csv', index=False)

Переименовываем столбцы:

In [7]:
df = df.rename(columns={'Категория': 'class_m', 'Производитель': 'ven_m', 'Название модели': 'model_m'})

Число записей до очистки:

In [8]:
dim_before = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 3421187


Статистики по распределению числа слов в наименовании модели:

In [9]:
print('Среднее число слов в моделе:', round(df['model_m'].str.split().apply(len).mean()))
print('Стандарт числа слов в моделе:', round(df['model_m'].str.split().apply(len).std()))

Среднее число слов в моделе: 3
Стандарт числа слов в моделе: 3


Показываем пропуски в относительных долях:

In [10]:
print('Число нулей для всех признаков:', dict(round(df.isnull().sum()/len(df), 2)))

Число нулей для всех признаков: {'class_m': 0.0, 'ven_m': 0.0, 'model_m': 0.0}


Показываем пропуски в абсолютных значения:

In [11]:
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'class_m': 0, 'ven_m': 0, 'model_m': 0}


Сброс записей с пропущенными значениями:

In [12]:
df.dropna(how='any', axis=0, inplace=True)

Удаление пунктуации из строковых атрибутов:

In [13]:
for item in tqdm(list(string.punctuation)):
    df[['class_m', 'model_m']] = df[['class_m', 'model_m']].applymap(lambda x: x.replace(item, ' '))

Убираем множественные пробелы и приводим к нижнему регистру:

In [14]:
df = df.applymap(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)
df = df.applymap(lambda x: x.strip().lower() if isinstance(x, str) else x)

Сброс дубликатов по вектору категория-производитель-модель:

In [15]:
df.drop_duplicates(inplace=True, keep='first', subset=['class_m', 'ven_m', 'model_m'])

Преобразование типов к строке:

In [16]:
df = df.astype({'class_m': 'str', 'ven_m': 'str', 'model_m': 'str'})

Выводим число нулей:

In [17]:
print('Число нулей для всех признаков:', dict(df.isnull().sum()))

Число нулей для всех признаков: {'class_m': 0, 'ven_m': 0, 'model_m': 0}


Удаляем дублирующиеся слова в названии модели:

In [18]:
df['model_m'] = df['model_m'].apply(lambda x: ' '.join(OrderedDict.fromkeys(x.split())))

Удаляем производителей из названий модели:

In [19]:
for item in tqdm(list(set(df['ven_m']))):
    df['model_m'] = df['model_m'].apply(lambda x: x.replace(item, ''))

Составляем список стоп слов русского словаря с отсечкой по длине: 

In [20]:
stop_words_ru = [item for item in get_stop_words('russian') if len(item) > min_word_length]

Удаляем стоп слова в названии модели:

In [21]:
for stop in tqdm(stop_words_ru):
    remove_stops = (lambda x: ' '.join([item for item in x.split() if item != stop]))
    df['model_m'] = df['model_m'].apply(remove_stops)

Создаем вектор ключевых слов по названиям категорий:

In [22]:
keys = list(" ".join(list(set(df['class_m']))).split())
keys = [x for x in keys if len(x) > min_word_length]

Вызываем классы для очистки и стемминга:

In [23]:
morph = pymorphy2.MorphAnalyzer()
stemmer = SnowballStemmer("russian")

Выделяем парадигмы ключевых слов:

In [24]:
keys = [morph.parse(item)[0].normal_form for item in keys]
keys = [stemmer.stem(item) for item in keys]

Оставляем слова в наименованиях модели, которые не содержат парадигмы: 

In [25]:
for key in tqdm(keys): 
    remove_keys = (lambda x: ' '.join([item for item in x.split() if (key not in item)]))
    df['model_m'] = df['model_m'].apply(remove_keys)

Оставляем только слова, которые отсутствуют в русском словаре Corpora:

In [26]:
remove_not_nouns = (lambda x: ' '.join([item for item in x.split() if morph.parse(item)[0].tag.POS is None]))
df['model_m'] = df['model_m'].apply(remove_not_nouns)

Удаляем характеристику объема памяти 'число + GB': она не влияет на tac номер:

In [27]:
df['model_m'] = df['model_m'].apply(lambda x: re.sub(r'\d+gb', '', x))

Отделяем цифры от не цифр:

In [28]:
df['model_m'] = df['model_m'].apply(lambda x: re.sub(r'(\D)(\d)', r'\1 \2', x))
df['model_m'] = df['model_m'].apply(lambda x: re.sub(r'(\d)(\D)', r'\1 \2', x))

Удаляем записи с пропущенными значениями:

In [29]:
df = df.apply(lambda x: x.str.strip() if isinstance(x, str) else x).replace('', np.nan)
df = df.dropna(how='any', axis=0)

Сброс дубликатов:

In [30]:
df.drop_duplicates(inplace=True, keep='first', subset=['class_m', 'ven_m', 'model_m'])

Статистики по распределению числа слов в наименовании модели:

In [31]:
print('Среднее число слов в моделе:', round(df['model_m'].str.split().apply(len).mean()))
print('Стандарт числа слов в моделе:', round(df['model_m'].str.split().apply(len).std()))

Среднее число слов в моделе: 5
Стандарт числа слов в моделе: 5


Число записей:

In [32]:
dim_after = int(df.shape[0])
print('Число записей:', int(df.shape[0]))

Число записей: 37041


Компрессия по числу записей:

In [33]:
print('Компрессия по числу записей:', round(dim_before/dim_after))

Компрессия по числу записей: 92


Описание фрейма:

In [34]:
df.describe()[:2][:]

,class_m,ven_m,model_m
count,37041,37041,37041
unique,76,1642,36343


Выводим случайный сэмпл:

In [35]:
df.sample(n=10)

,class_m,ven_m,model_m
566,радар детекторы,inspector,rd s 1
7175,принтеры и мфу,canon,ierunner 2530 i
8946,электронные книги,digma,e 6
12700,моноблоки,hp,23 8 24 f 0042 ur 4 gt 06 ea
162,автомагнитолы,swat,mex 1027 ubw
1193,радиоприемники,vitek,wx 4052
7539,комплекты акустики,harman/kardon,sb 30
6425,мобильные телефоны,alcatel,one touch 2012 d
2726,сумки чехлы для фото и видеотехники,baohwa,6085
8281,электронные книги,qumo,libro classic


Записываем csv в директорию:

In [36]:
df.to_csv('data/market_clean.csv', index=False)